In [14]:
import json
import os
import re
from google.colab import drive

# 1. Drive'ı Bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Fonksiyon Tanımı (Hata almamak için burada tanımlıyoruz)
def validate_synthetic_data(json_data):
    report = []
    for idx, item in enumerate(json_data):
        q_id = f"Soru {idx + 1}"
        error_found = False
        logs = []

        # Şık ve Rationale Tutarlılığı (Harf kontrolü)
        rationale_text = item.get("rationale", "")
        correct_letter = item.get("correct", "")

        rationale_match = re.search(r"Doğru seçenek ([A-E])", rationale_text)
        if rationale_match:
            if rationale_match.group(1) != correct_letter:
                logs.append(f"HARF HATASI: Çözüm {rationale_match.group(1)} diyor, anahtar {correct_letter}.")
                error_found = True

        # Sayısal Değer Çakışması
        options = item.get("options", [])
        correct_option_text = next((opt for opt in options if opt.startswith(correct_letter)), "")

        # Rationale içindeki sayıları bul (Ondalıklı sayılar dahil)
        numbers = re.findall(r"[-+]?\d*\.\d+|\d+", rationale_text)
        if numbers and correct_option_text:
            last_calc_value = numbers[-1]
            if last_calc_value not in correct_option_text:
                logs.append(f"DEĞER HATASI: Çözümdeki sayı ({last_calc_value}) şıkta bulunamadı.")
                error_found = True

        if error_found:
            report.append({"id": q_id, "errors": logs})
    return report

# 3. Dosya Listesi ve İşleme
file_list = [
   # '/content/drive/MyDrive/Proje_Yedekleri/mat_train.json',
   # '/content/drive/MyDrive/Proje_Yedekleri/mat_test.json',
    #'/content/drive/MyDrive/Proje_Yedekleri/fizik_train.json',
    #'/content/drive/MyDrive/Proje_Yedekleri/fizik_test.json',
    #'/content/drive/MyDrive/Proje_Yedekleri/kimya_train.json',
    #'/content/drive/MyDrive/Proje_Yedekleri/kimya_test.json'
   '/content/drive/MyDrive/Proje_Yedekleri/kimya_dogru_test.json'

]

results = {}

print("🚀 Doğrulama işlemi başlıyor...\n")
for path in file_list:
    name = os.path.basename(path)
    if os.path.exists(path):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                errors = validate_synthetic_data(data)
                if errors:
                    results[name] = errors
                    print(f"❌ {name}: {len(errors)} hatalı soru bulundu.")
                else:
                    print(f"✅ {name}: Temiz.")
        except Exception as e:
            print(f"❗ {name} okunurken hata oluştu: {e}")
    else:
        print(f"⚠️ {name} bulunamadı, yolunuzu kontrol edin.")

# 4. Hata Özeti
if results:
    print("\n--- 📝 HATA DETAYLARI ---")
    print(json.dumps(results, indent=2, ensure_ascii=False))

🚀 Doğrulama işlemi başlıyor...

❌ kimya_dogru_test.json: 33 hatalı soru bulundu.

--- 📝 HATA DETAYLARI ---
{
  "kimya_dogru_test.json": [
    {
      "id": "Soru 1",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (4) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 2",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (3) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 3",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (5) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 4",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (7) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 5",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (107) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 6",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (180) şıkta bulunamadı."
      ]
    },
    {
      "id": "Soru 7",
      "errors": [
        "DEĞER HATASI: Çözümdeki sayı (2) şıkta bulunamadı."
      ]
    },
    

In [ ]:
def extract_faulty_questions(file_list):
    for path in file_list:
        name = os.path.basename(path)
        if os.path.exists(path):
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Hataları bul
            errors = validate_synthetic_data(data)
            # Hatalı soruların indekslerini al
            error_indices = [int(e['id'].split(" ")[1]) - 1 for e in errors]

            # Sadece hatalı olanları filtrele
            faulty_data = [item for i, item in enumerate(data) if i in error_indices]

            if faulty_data:
                error_file_path = path.replace(".json", "_HATALI_SORULAR.json")
                with open(error_file_path, 'w', encoding='utf-8') as f:
                    json.dump(faulty_data, f, ensure_ascii=False, indent=2)
                print(f"📁 {name} -> {len(faulty_data)} hatalı soru ayıklandı: {os.path.basename(error_file_path)}")
            else:
                print(f"✅ {name} dosyasında ayıklanacak hata bulunamadı.")

# Hatalı soruları ayıkla
extract_faulty_questions(file_list)

📁 mat_train.json -> 16 hatalı soru ayıklandı: mat_train_HATALI_SORULAR.json
📁 mat_test.json -> 5 hatalı soru ayıklandı: mat_test_HATALI_SORULAR.json
📁 fizik_train.json -> 53 hatalı soru ayıklandı: fizik_train_HATALI_SORULAR.json
📁 fizik_test.json -> 21 hatalı soru ayıklandı: fizik_test_HATALI_SORULAR.json
📁 kimya_train.json -> 106 hatalı soru ayıklandı: kimya_train_HATALI_SORULAR.json
📁 kimya_test.json -> 34 hatalı soru ayıklandı: kimya_test_HATALI_SORULAR.json


DÜZELTİLMİŞ VERİLERİN ANA DOSYAYA KAYDI

In [7]:
import json
import os

# 1. Birleştirme Fonksiyonu
def merge_fixed_questions(original_path, fixed_path, final_output_path):
    if not os.path.exists(original_path) or not os.path.exists(fixed_path):
        print(f"⚠️ Dosya eksik: {os.path.basename(original_path)} veya {os.path.basename(fixed_path)}")
        return

    with open(original_path, 'r', encoding='utf-8') as f:
        original_data = json.load(f)

    with open(fixed_path, 'r', encoding='utf-8') as f:
        fixed_data = json.load(f)

    # Soru metni üzerinden bir eşleştirme sözlüğü oluşturuyoruz
    fixed_dict = {item['question'].strip(): item for item in fixed_data}

    final_data = []
    merged_count = 0

    for item in original_data:
        q_text = item['question'].strip()
        if q_text in fixed_dict:
            # Hatalı olanın yerine AI'dan gelen düzeltilmiş versiyonu koy
            final_data.append(fixed_dict[q_text])
            merged_count += 1
        else:
            # Zaten doğru olan (ayıklanmamış) soru kalsın
            final_data.append(item)

    with open(final_output_path, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

    print(f"✅ {os.path.basename(original_path)}: {merged_count} soru güncellendi -> {os.path.basename(final_output_path)}")

# 2. Dosya Listesi ve Döngü
# Dosya yollarını kendi Drive klasör yapına göre güncellemelisin
base_path = '/content/drive/MyDrive/Proje_Yedekleri/'

files_to_process = [
    #('mat_train.json', 'mat_train_DUZELTILMIS.json', 'mat_train_FINAL.json'),
    #('mat_test.json', 'mat_test_DUZELTILMIS.json', 'mat_test_FINAL.json'),
    #('fizik_train.json', 'fizik_train_DUZELTILMIS.json', 'fizik_train_FINAL.json'),
    #('fizik_test.json', 'fizik_test_DUZELTILMIS.json', 'fizik_test_FINAL.json'),
    ('kimya_train.json', 'kimya_train_DUZELTILMIS.json', 'kimya_train_FINAL.json'),
    #('kimya_test.json', 'kimya_test_DUZELTILMIS.json', 'kimya_test_FINAL.json')
]

print("🔄 Veri setleri birleştiriliyor...\n")
for orig, fixed, final in files_to_process:
    merge_fixed_questions(base_path + orig, base_path + fixed, base_path + final)

🔄 Veri setleri birleştiriliyor...

✅ kimya_train.json: 125 soru güncellendi -> kimya_train_FINAL.json
